In [1]:
# При изменениях в коде проекте эти обновления будут автоматически подгружаться в ноутбук
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import inspect
import argparse
import numpy as np
from detox import DeToxEdit
import torch

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)

from utils.startup import main
from evaluation.evaluate_model import evaluate_model

if '__file__' in globals():
    parser = argparse.ArgumentParser(description='DeTox')
    parser.add_argument('--config_file', default='gpt2-medium.ini', type=str, help='Config Filename. E.g. gpt2-medium.ini')
    args = parser.parse_args()
    config_filename = args.config_file
else:
    config_filename = 'gpt2-medium.ini'

config = main(config_filename=config_filename)


/Users/ellina/Documents/Projection Filter for Subspaces/detox-edit/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/ellina/Documents/Projection Filter for Subspaces/detox-edit/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


No CUDA_VISIBLE_DEVICES found. Setting to 0.
PROJECT_ROOT: 'detox-edit'
PYTHONPATH: 'detox-edit':
HF_HOME: 'detox-edit'


In [3]:
from utils.model_utils import load_large_model
from detox import DeToxEdit

model_id = 'gpt2'
model, tokenizer = load_large_model(model_id)

INFO:root:Model gpt2 loaded.


In [4]:
lower_layer = 10
upper_layer = 24

layer_range = np.arange(lower_layer, upper_layer) if lower_layer != -1 and upper_layer != -1 else None

In [5]:
import json
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import logging

In [ ]:
# Пути к данным для доменов
data_paths = {
    #"toxicity": "toxicity_pairwise/toxicity.jsonl",
    #"sentiment": "toxicity_pairwise/sentiment.jsonl",
    "clickbait": "toxicity_pairwise/clickbait.jsonl"
}

# Словарь для хранения результатов
results = {}

# Обрабатываем данные для каждого домена
for domain, path in data_paths.items():
    print(f"Обрабатываем домен: {domain}")
    
    # Без центрирования
    editor_no_center = editor = DeToxEdit(
    model=model,
    tokenizer=tokenizer,
    pref_data_dps=313,
    centering=False,
    top_k_ranks=2,
    edit_layer_range=layer_range,
    random_dps=True
)
    not_center_ats, not_center_pref_matrix = editor_no_center.get_ats(path)
    
    # С центрированием
    editor_center = DeToxEdit(
    model=model,
    tokenizer=tokenizer,
    pref_data_dps=313,
    centering=True,
    top_k_ranks=2,
    edit_layer_range=layer_range,
    random_dps=True
)
    center_ats, center_pref_matrix = editor_center.get_ats(path)
    
    # Сохраняем результаты
    results[domain] = {
        "not_center_ats": not_center_ats,
        "not_center_pref_matrix": not_center_pref_matrix,
        "center_ats": center_ats,
        "center_pref_matrix": center_pref_matrix
    }

INFO:root:Loaded 313 preferred and 313 non-preferred samples.


Обрабатываем домен: clickbait


INFO:root:Batch 1/6 of size 50


KeyboardInterrupt: 

In [ ]:
def pca_visualize_preference_matrix(pref_matrix, title):
    """
    Применяет PCA к матрице предпочтений и строит график.
    Args:
        pref_matrix (torch.Tensor): Матрица предпочтений (L, N, D).
        title (str): Заголовок графика.
    """
    from sklearn.decomposition import PCA

    # Конкатенируем все слои
    all_layers = pref_matrix.cpu().numpy().reshape(-1, pref_matrix.shape[-1])  # (L*N, D)
    
    # PCA
    pca = PCA(n_components=2)
    reduced = pca.fit_transform(all_layers)
    
    # Визуализация
    plt.figure(figsize=(8, 6))
    plt.scatter(reduced[:, 0], reduced[:, 1], s=2, alpha=0.5)
    plt.title(title)
    plt.xlabel("PCA Component 1")
    plt.ylabel("PCA Component 2")
    plt.show()

# Визуализация для всех доменов
for domain, data in results.items():
    print(f"Визуализация для домена: {domain}")
    
    # Без центрирования
    pca_visualize_preference_matrix(data["not_center_pref_matrix"], 
                                    title=f"{domain.capitalize()} - Preference Matrix (без центрирования)")
    
    # С центрированием
    pca_visualize_preference_matrix(data["center_pref_matrix"], 
                                    title=f"{domain.capitalize()} - Preference Matrix (с центрированием)")
